In [30]:
import sklearn.model_selection
import sklearn.datasets
import sklearn.metrics
import pandas as pd
import autosklearn.classification
import os
import featuretools as ft
import warnings
warnings.filterwarnings('ignore')

In [34]:
def run_as(df, target_ft):
    try:
        os.remove('/tmp/autosklearn_regression_example_tmp')
        os.remove('/tmp/autosklearn_regression_example_out')
    except:
        pass
    X = df.drop(columns=target_ft)
    y = df[target_ft]
    X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, random_state=1)
    automl = autosklearn.classification.AutoSklearnClassifier(
        time_left_for_this_task=30,
        per_run_time_limit=30,
        tmp_folder='./tmp/autosklearn_regression_example_tmp',
        output_folder='./tmp/autosklearn_regression_example_out',
    )
    automl.fit(X_train, y_train)
    y_hat = automl.predict(X_test)
    return "Accuracy score - " + str(sklearn.metrics.accuracy_score(y_test, y_hat))
    
    
def gen_feats(df):
    es = ft.EntitySet(id = 'df')
    es.entity_from_dataframe(entity_id = 'data', dataframe = df, 
                         make_index = True, index = 'index')
    feature_matrix, feature_defs = ft.dfs(entityset = es, target_entity = 'data',
                                      trans_primitives = ['add_numeric', 'multiply_numeric'])
    return feature_matrix

def run_test(df,target_ft, mode = 0):
    results = []
    if mode!= 1:
        rs = run_as(df,target_ft)   
        results.append("Autosk Only: " + rs)
    if mode!=0:
        print("")
        df = gen_feats(df)
        rs = run_as(df,target_ft)
        results.append("Autosk with Featuretools: " + rs)
    
    print("===================================")
    [print(x) for x in results]

In [35]:
df = pd.read_csv("blood.csv")
target_ft = "class"
run_test(df, target_ft, mode=2)

[WARNING] [2020-07-12 23:06:49,438:AutoML(1):828fa7cf24a4202d3c29fc1fbc4b82b7] Time limit for a single run is higher than total time limit. Capping the limit for a single run to the total time given to SMAC (29.598984)
[WARNING] [2020-07-12 23:06:49,442:AutoML(1):828fa7cf24a4202d3c29fc1fbc4b82b7] Capping the per_run_time_limit to 14.0 to have time for a least 2 models in each process.
[WARNING] [2020-07-12 23:06:53,480:EnsembleBuilder(1):828fa7cf24a4202d3c29fc1fbc4b82b7] No models better than random - using Dummy Score!Number of models besides current dummy model: 1. Number of dummy models: 1
[WARNING] [2020-07-12 23:06:55,493:EnsembleBuilder(1):828fa7cf24a4202d3c29fc1fbc4b82b7] No models better than random - using Dummy Score!Number of models besides current dummy model: 5. Number of dummy models: 1
[WARNING] [2020-07-12 23:06:57,508:EnsembleBuilder(1):828fa7cf24a4202d3c29fc1fbc4b82b7] No models better than random - using Dummy Score!Number of models besides current dummy model: 7. Nu